# HCP tracer calculation
What follows is the calculation of the tracer correlation for an HCP crystal (ideal $c/a = \sqrt{8/3}$, $a_0 = 1$, and $\nu = 1$, all for convenience).

In [1]:
import sys
sys.path.append('../')
import numpy as np
import yaml
from onsager import OnsagerCalc
from onsager import crystal
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
HCP = crystal.Crystal.HCP(1., chemistry="ideal HCP")
print(HCP)

#Lattice:
  a1 = [ 0.5       -0.8660254  0.       ]
  a2 = [0.5       0.8660254 0.       ]
  a3 = [0.         0.         1.63299316]
#Basis:
  (ideal HCP) 0.0 = [0.33333333 0.66666667 0.25      ]
  (ideal HCP) 0.1 = [0.66666667 0.33333333 0.75      ]


In [3]:
sitelist = HCP.sitelist(0)
vacancyjumps = HCP.jumpnetwork(0, 1.01)
for jlist in vacancyjumps:
    print('---')
    for (i,j), dx in jlist:
        print(i, '-', j, dx)

---
1 - 1 [-0.5        0.8660254  0.       ]
1 - 1 [ 0.5       -0.8660254 -0.       ]
0 - 0 [ 0.5       -0.8660254  0.       ]
0 - 0 [-0.5        0.8660254 -0.       ]
0 - 0 [-1.00000000e+00  5.55111512e-17  0.00000000e+00]
0 - 0 [ 1.00000000e+00 -5.55111512e-17 -0.00000000e+00]
1 - 1 [1.00000000e+00 5.55111512e-17 0.00000000e+00]
1 - 1 [-1.00000000e+00 -5.55111512e-17 -0.00000000e+00]
1 - 1 [-0.5       -0.8660254  0.       ]
1 - 1 [ 0.5        0.8660254 -0.       ]
0 - 0 [0.5       0.8660254 0.       ]
0 - 0 [-0.5       -0.8660254 -0.       ]
---
1 - 0 [0.         0.57735027 0.81649658]
0 - 1 [-0.         -0.57735027 -0.81649658]
0 - 1 [ 0.         -0.57735027  0.81649658]
1 - 0 [-0.          0.57735027 -0.81649658]
0 - 1 [-0.5         0.28867513 -0.81649658]
1 - 0 [ 0.5        -0.28867513  0.81649658]
1 - 0 [-0.5        -0.28867513 -0.81649658]
0 - 1 [0.5        0.28867513 0.81649658]
1 - 0 [-0.5        -0.28867513  0.81649658]
0 - 1 [ 0.5         0.28867513 -0.81649658]
1 - 0 [ 0.5 

In [4]:
HCPdiffuser = OnsagerCalc.VacancyMediated(HCP, 0, sitelist, vacancyjumps, 1)

In [5]:
for state in HCPdiffuser.interactlist():
    print(state)

1.[0,0,0]:0.[1,0,0] (dx=[0.5,-0.28867513459481287,-0.8164965809277259])
0.[0,0,0]:0.[0,-1,0] (dx=[-0.5,-0.8660254037844386,-0.0])


In [6]:
nu0 = 1
dE0 = 1
HCPtracer = {'preV': np.ones(1), 'eneV': np.zeros(1), 
             'preT0': np.array([nu0, nu0]), 'eneT0': np.array([dE0, dE0])}
HCPtracer.update(HCPdiffuser.maketracerpreene(**HCPtracer))
for k,v in zip(HCPtracer.keys(), HCPtracer.values()): print(k,v)

preV [1.]
eneV [0.]
preT0 [1 1]
eneT0 [1 1]
preS [1.]
eneS [0.]
preSV [1. 1.]
eneSV [0. 0.]
preT1 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
eneT1 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
preT2 [1. 1.]
eneT2 [1. 1.]


In [7]:
Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(1, **HCPtracer))

Correlation coefficient = $L_\text{ss} / L_\text{sv}$ (as $L_\text{sv} = L_\text{vv}$). Should be very close to the FCC correlation coefficient of 0.78145, for this purely isotropic diffusion case.

In [8]:
np.dot(Lss, np.linalg.inv(Lsv))

array([[-0.78120488,  0.        ,  0.        ],
       [ 0.        , -0.78120488,  0.        ],
       [ 0.        ,  0.        , -0.78145142]])

In [9]:
print(HCPdiffuser.GFvalues)

{vacancyThermoKinetics(pre=[1.], betaene=[0.], preT=[1. 1.], betaeneT=[1. 1.]): array([-0.30459761, -0.07807413, -0.07807413, -0.05208619, -0.04717251,
       -0.04427807, -0.04427807, -0.04427807, -0.04004718, -0.03871072,
       -0.03390637, -0.03199645, -0.03199645, -0.03134279, -0.03052154,
       -0.02955812, -0.02898466, -0.02898466, -0.02898466, -0.02826905,
       -0.02651933, -0.02539044, -0.02559087, -0.0245483 , -0.0245483 ,
       -0.02415435, -0.02374109, -0.02349613, -0.02312376, -0.02271259,
       -0.02243781, -0.02237242, -0.02210375, -0.02210375, -0.02175881,
       -0.02124551, -0.02124551, -0.02124551, -0.0206982 , -0.0206982 ,
       -0.01997178, -0.01915028])}


In [10]:
print(yaml.dump(HCPdiffuser.GFvalues))

? !VacancyThermoKinetics
  betaene: !numpy.ndarray [0.0]
  betaeneT: !numpy.ndarray [1.0, 1.0]
  pre: !numpy.ndarray [1.0]
  preT: !numpy.ndarray [1.0, 1.0]
: !numpy.ndarray [-0.30459761217407005, -0.07807412648137282, -0.07807412629113923,
  -0.05208618893058945, -0.04717250692903148, -0.0442780743241831, -0.04427807367034695,
  -0.04427807367034695, -0.04004718239555193, -0.03871072296928411, -0.033906365489048074,
  -0.031996448148095, -0.031996448148095, -0.031342790825141395, -0.030521536490081286,
  -0.029558119626874765, -0.02898466441122912, -0.02898466269132974, -0.02898466269132974,
  -0.028269045469956632, -0.02651932953247783, -0.0253904356141744, -0.025590867621937047,
  -0.02454830076121832, -0.02454830076121832, -0.024154353318393343, -0.023741094014366425,
  -0.023496126022377817, -0.02312375820273484, -0.022712592888975043, -0.022437807548903437,
  -0.022372421954471845, -0.022103747960451742, -0.022103747960451742, -0.02175880593490824,
  -0.021245513841386468, -0.021